# Data-Set Secondary Pre-Processing

This notebook further focusses on processing steps we can use to enhance the images in our data-sets

## Author: Alexander Goudemond, Student Number: 219030365

# Imports

# Previous Results:

This section of the notebook summarises the notes from 004

1105, 1106, 1201 and 1202 are useful to show the user the hidden information in the images. 

0801 and 0802 are super close at being useful, however, they posses undesirable features

0805, 1101 and 1103 may be useful for segmentation

It may be worth attempting to try introduce an AND gate between 2 images, in order to extract only the common information. Though this may preserve noise... 

This notebook appears to be rather unhelpful, however we have exhausted possibilities for Image Processing. To get an idea of what we are hoping to achieve, let us open 2 images from one of our reference materials: Magnussen (PhD Thesis)

The images will be available in the COMP700 Proposal, if not provided by the author



Segmentation Example


![Segmentation Example](Magnusson_Segmentation_Sample.png "Segmentation Example")

Tracking Example Below:

![Tracking Example](Magnusson_Track_Sample.png "Tracking Example")

As we can see above, in the segmentation example from Magnussen, (b) is a desirable Processing step before segmentation

At the moment, we have the following desirable affects:


Hidden Info present (1105, 1106, 1201, 1202):


![Hidden Info 1](004_InitialPreProcessing\1105_ErodedNegativeImages.png "Hidden Info 1")

![Hidden Info 2](004_InitialPreProcessing\1106_OpenedNegativeImages.png "Hidden Info 2")

![Hidden Info 3](004_InitialPreProcessing\1201_HistEq_ErodedAtEnd_BlurAdaptiveThresholdMean.png "Hidden Info 3")

![Hidden Info 4](004_InitialPreProcessing\1202_HistE1_OpenedAtStart_BlurAdaptiveThresholdMean.png "Hidden Info 4")

Almost pleasant processed Images (0801, 0802):

![Almost Pleasant Processed Images 1](004_InitialPreProcessing\0801_OtsuThreshold.png "Almost Pleasant Processed Images 1")

![Almost Pleasant Processed Images 2](004_InitialPreProcessing\0802_Blur_OtsuThreshold.png "Almost Pleasant Processed Images 2")


Potential Useful Images (0805, 1101, 1103):


![Potentially Useful Images 1](004_InitialPreProcessing\0805_BlurAdaptiveThresholdMean.png "Potentially Useful Images 1")

![Potentially Useful Images 2](004_InitialPreProcessing\1101_ErodedImages.png "Potentially Useful Images 2")

![Potentially Useful Images 3](004_InitialPreProcessing\1103_OpenedImages.png "Potentially Useful Images 3")

As we can see from the 3 pictures above, The processing we are attempting to produce is not of the same quality of Magnussen YET

One of the challenges we are facing is to find a processing step that assists all 10 data-sets, because if we only focussed on 1 data-set we would be able to pick a specific processing option... 

For ease of reference, bottom left of the images above corresponds to Magnussen:

![Segmentation Example](Magnusson_Segmentation_Sample.png "Segmentation Example")

From the above images, we can visualy see that 1103 is the closest to improving things, but it is rather dark. We need to try use this technique to brighten the overall image, so that we can move forward. The other techniques tried in 004 may be unhelpful as they either help a few data-sets and damage others, or introduce noise...

We will carefully explore the combinations here, though